# Import Libraries

In [1]:
import requests
from requests.exceptions import HTTPError
import pandas as pd
import json as js
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
# Uncomment this if you like to use the old MPL library
import matplotlib.dates as mpl_dates
import matplotlib.ticker as tkr
%matplotlib inline
import seaborn as sns
#technical indicator library
import btalib
#from mpl_finance import candlestick_ohlc
import mplfinance as mpf

# Import Data with CoinbasePro API

In [2]:
import cbpro
public_client = cbpro.PublicClient()


In [3]:
public_client.get_products()

[{'id': 'REP-USD',
  'base_currency': 'REP',
  'quote_currency': 'USD',
  'base_min_size': '0.10000000',
  'base_max_size': '5000.00000000',
  'quote_increment': '0.01000000',
  'base_increment': '0.00000100',
  'display_name': 'REP/USD',
  'min_market_funds': '10',
  'max_market_funds': '30000',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'trading_disabled': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'NMR-USD',
  'base_currency': 'NMR',
  'quote_currency': 'USD',
  'base_min_size': '0.01000000',
  'base_max_size': '3900.00000000',
  'quote_increment': '0.00010000',
  'base_increment': '0.00100000',
  'display_name': 'NMR/USD',
  'min_market_funds': '1.0',
  'max_market_funds': '100000',
  'margin_enabled': False,
  'post_only': False,
  'limit_only': False,
  'cancel_only': False,
  'trading_disabled': False,
  'status': 'online',
  'status_message': ''},
 {'id': 'ALGO-BTC',
  'base_currency': 'ALGO',
  'quote_cu

In [4]:
# Get the order book at the default level.
public_client.get_product_order_book('BTC-USD')
# Get the order book at a specific level.
public_client.get_product_order_book('BTC-USD', level=1)

{'bids': [['45197.07', '0.05071303', 1]],
 'asks': [['45199.94', '0.21300938', 3]],
 'sequence': 21172371165}

In [5]:
# Get the product ticker for a specific product.
public_client.get_product_ticker(product_id='BTC-USD')

{'trade_id': 131648363,
 'price': '45199.94',
 'size': '0.09749724',
 'time': '2021-02-10T22:36:36.776441Z',
 'bid': '45195.99',
 'ask': '45199.94',
 'volume': '26356.57001566'}

In [6]:
# Get the product trades for a specific product.
# Returns a generator
public_client.get_product_trades(product_id='BTC-USD')

<generator object PublicClient._send_paginated_message at 0x7fec74481ca8>

In [7]:
public_client.get_product_24hr_stats('BTC-USD')

{'open': '46906.33',
 'high': '47349.99',
 'low': '43706.86',
 'volume': '26356.57001566',
 'last': '45171.99',
 'volume_30day': '896801.20722831'}

In [8]:
public_client.get_product_historic_rates('BTC-USD')
# To include other parameters, see function docstring:
data = public_client.get_product_historic_rates('BTC-USD', granularity=21600)
data

[[1612980000, 44380, 45292, 45050.01, 45175.47, 4278.56583297],
 [1612958400, 43706.86, 46659.75, 46659.64, 45050, 11251.5032324],
 [1612936800, 46168.92, 47349.99, 46180.31, 46662.46, 3365.98910448],
 [1612915200, 45666, 46966.9, 46514.73, 46181.15, 6096.98783704],
 [1612893600, 46289, 47558.18, 46559.42, 46514.73, 7203.37867622],
 [1612872000, 45020, 47150, 46453.41, 46562, 8743.75929171],
 [1612850400, 45606.34, 48200, 46949.44, 46445.69, 7583.87494091],
 [1612828800, 45567, 47550, 46453.66, 46949.46, 12887.78074232],
 [1612807200, 42573.57, 46750, 43148.66, 46448.1, 14620.11644118],
 [1612785600, 39155.02, 44900, 39475, 43145.49, 27473.64555827],
 [1612764000, 38642.9, 39540, 38703.08, 39462.54, 2294.47704357],
 [1612742400, 38057.01, 39190.56, 38871.43, 38701.98, 3545.77134528],
 [1612720800, 37760.48, 39150, 37913.57, 38871.42, 4131.35876772],
 [1612699200, 37371.35, 39315.72, 39088.24, 37913.56, 6470.19473948],
 [1612677600, 38317.02, 39737.83, 38547.96, 39099.29, 3532.20804296]

In [9]:
#keep what we need from the obtain data and that is the date, high, low, open, close
for line in data:
    del line[5:]

In [11]:
df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
df.set_index('date', inplace=True)
df

,open,high,low,close
date,,,,
1612980000,44380.00,45292.00,45050.01,45175.47
1612958400,43706.86,46659.75,46659.64,45050.00
1612936800,46168.92,47349.99,46180.31,46662.46
1612915200,45666.00,46966.90,46514.73,46181.15
1612893600,46289.00,47558.18,46559.42,46514.73
...,...,...,...,...
1606608000,17535.26,17900.00,17736.10,17807.76
1606586400,17652.00,17933.25,17770.23,17739.85
1606564800,17071.71,17789.00,17166.00,17768.94


In [12]:
df['20sma'] = df.close.rolling(20).mean()
df

,open,high,low,close,20sma
date,,,,,
1612980000,44380.00,45292.00,45050.01,45175.47,NaN
1612958400,43706.86,46659.75,46659.64,45050.00,NaN
1612936800,46168.92,47349.99,46180.31,46662.46,NaN
1612915200,45666.00,46966.90,46514.73,46181.15,NaN
1612893600,46289.00,47558.18,46559.42,46514.73,NaN
...,...,...,...,...,...
1606608000,17535.26,17900.00,17736.10,17807.76,18919.2095
1606586400,17652.00,17933.25,17770.23,17739.85,18833.7700
1606564800,17071.71,17789.00,17166.00,17768.94,18756.3655


In [13]:
sma = btalib.sma(df.close)
print(sma.df)

                     sma
date                    
1612980000           NaN
1612958400           NaN
1612936800           NaN
1612915200           NaN
1612893600           NaN
...                  ...
1606608000  18954.682333
1606586400  18909.344000
1606564800  18862.809667
1606543200  18796.326333
1606521600  18724.647000

[300 rows x 1 columns]


In [17]:
df['sma'] = btalib.sma(df.close, period=20).df
df

,open,high,low,close,20sma,sma
date,,,,,,
1612980000,44380.00,45292.00,45050.01,45175.47,NaN,NaN
1612958400,43706.86,46659.75,46659.64,45050.00,NaN,NaN
1612936800,46168.92,47349.99,46180.31,46662.46,NaN,NaN
1612915200,45666.00,46966.90,46514.73,46181.15,NaN,NaN
1612893600,46289.00,47558.18,46559.42,46514.73,NaN,NaN
...,...,...,...,...,...,...
1606608000,17535.26,17900.00,17736.10,17807.76,18919.2095,18919.2095
1606586400,17652.00,17933.25,17770.23,17739.85,18833.7700,18833.7700
1606564800,17071.71,17789.00,17166.00,17768.94,18756.3655,18756.3655
